In [1]:
import os
import pandas as pd
import json
import numpy as np


In [2]:
prj_root = os.getcwd()
dataset_folderpath= os.path.join(prj_root,'mitre_nav_reports')
ttp_list_corpus=[]

In [3]:
for root, dirs, files in os.walk(dataset_folderpath):
    for filename in files:
        abs_filepath = os.path.join(root,filename)
        # print(abs_filepath)
        with open(abs_filepath,'r') as f:
            data = data = json.load(f)

        df = pd.DataFrame(data['techniques'])
        # df = df[df['score'].notna()]
        df = df.loc[df['score'].isin([1.0])]
        ttp_list= df['techniqueID'].tolist()
        for ttp in ttp_list:
            ttp_list_corpus.append(ttp)
        
        # ttp_list_corpus.append(' '.join(ttp_list))
    
#     break

In [4]:
# src:https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
from sklearn.feature_extraction.text import CountVectorizer
#instantiate CountVectorizer() 
cv=CountVectorizer(analyzer=lambda d: d.split(' ')) 
word_count_vector=cv.fit_transform(ttp_list_corpus)

In [5]:
#freq_count of all ttps
print(dict(zip(cv.get_feature_names(),word_count_vector.toarray().sum(axis=0))))

{'T1001': 5, 'T1001.001': 11, 'T1001.002': 8, 'T1001.003': 11, 'T1003': 6, 'T1003.001': 13, 'T1003.002': 7, 'T1003.004': 1, 'T1003.005': 1, 'T1005': 92, 'T1007': 38, 'T1008': 39, 'T1010': 25, 'T1011.001': 1, 'T1012': 71, 'T1014': 19, 'T1016': 158, 'T1016.001': 5, 'T1018': 32, 'T1020': 13, 'T1021': 2, 'T1021.001': 12, 'T1021.002': 20, 'T1021.003': 1, 'T1021.004': 2, 'T1021.005': 5, 'T1021.006': 1, 'T1025': 18, 'T1027': 226, 'T1027.001': 23, 'T1027.002': 56, 'T1027.003': 17, 'T1027.004': 4, 'T1027.005': 8, 'T1027.006': 1, 'T1029': 14, 'T1030': 9, 'T1033': 130, 'T1036': 23, 'T1036.001': 4, 'T1036.003': 1, 'T1036.004': 39, 'T1036.005': 88, 'T1036.006': 1, 'T1036.007': 1, 'T1037.001': 4, 'T1037.004': 4, 'T1037.005': 1, 'T1039': 4, 'T1040': 5, 'T1041': 81, 'T1046': 21, 'T1047': 55, 'T1048': 6, 'T1048.003': 14, 'T1049': 45, 'T1052.001': 5, 'T1053': 2, 'T1053.002': 1, 'T1053.003': 10, 'T1053.005': 71, 'T1055': 40, 'T1055.001': 40, 'T1055.002': 5, 'T1055.003': 3, 'T1055.004': 6, 'T1055.005': 1,

c:\users\yi sheng\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# tfidf_transformer.idf_

TfidfTransformer()

In [7]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [8]:
# you only needs to do this once
feature_names=cv.get_feature_names()

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([' '.join(ttp_list_corpus)]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,len(feature_names))


for k in keywords:
    print(k,keywords[k])

T1105 0.295
T1082 0.276
T1071.001 0.247
T1027 0.247
T1059.003 0.234
T1083 0.233
T1057 0.209
T1070.004 0.2
T1140 0.195
T1016 0.186
T1547.001 0.182
T1033 0.16
T1573.001 0.145
T1106 0.141
T1113 0.14
T1112 0.13
T1056.001 0.123
T1005 0.121
T1036.005 0.116
T1543.003 0.111
T1041 0.109
T1518.001 0.1
T1059.001 0.1
T1053.005 0.098
T1012 0.098
T1132.001 0.095
T1027.002 0.08
T1047 0.079
T1204.002 0.078
T1074.001 0.078
T1124 0.077
T1562.001 0.075
T1218.011 0.073
T1059.005 0.073
T1095 0.071
T1049 0.067
T1555.003 0.066
T1573.002 0.064
T1055.001 0.061
T1055 0.061
T1036.004 0.059
T1008 0.059
T1007 0.058
T1566.001 0.057
T1486 0.055
T1497.001 0.054
T1564.001 0.053
T1569.002 0.052
T1497.003 0.052
T1553.002 0.05
T1120 0.05
T1018 0.05
T1560 0.049
T1087.001 0.048
T1070.006 0.048
T1548.002 0.046
T1490 0.046
T1135 0.046
T1119 0.043
T1115 0.043
T1102.002 0.043
T1059.004 0.043
T1574.002 0.042
T1560.003 0.042
T1090 0.042
T1071.004 0.042
T1547.009 0.041
T1489 0.041
T1010 0.041
T1070 0.039
T1036 0.038
T1027.001 0.0

In [9]:
df1 = pd.DataFrame([keywords])
df1
df1.to_csv (r'tf-tdf.csv', index = False, header=True)